In [1]:
#import os
#os.chdir('D:\\analytics edge\\gurobi')

In [2]:
def optimize(excel):
    from gurobipy import GRB, Model
    import pandas as pd
    #Extract the data from different sheets in the excel
    FC=pd.read_excel(excel,sheet_name='Fulfilment Centers',index_col=0)
    R=pd.read_excel(excel,sheet_name='Regions',index_col=0)
    Dis=pd.read_excel(excel,sheet_name='Distances',index_col=0)
    Item=pd.read_excel(excel,sheet_name='Items',index_col=0)
    De=pd.read_excel(excel,sheet_name='Demand',index_col=0)
    #Build the optimization model
    mod=Model()
    I=FC.index
    K=Item.index
    J=R.index
    x=mod.addVars(I,J,K,name='x')
    # Objective: Minimize the transportation cost.
    #            Summing all the item of each Fulfillment Center that satisfies different demand in each region
    # multiplies 1.38, weight of each item and the distance from Fulfillment Center i to Region j
    mod.setObjective(sum(x[i,j,k]*1.38*Item.loc[k,"shipping_weight"]*Dis.loc[j,i] for i in I for j in J for k in K))
    # Constraint 1: for each region j and each item k, the sum of the items from each Fulfillment Center i should meet
    # the demand of it.
    for j in J:
        for k in K:
            mod.addConstr(sum(x[i,j,k] for i in I)>=De.loc[k,j])
    # Constraint 2: for each Fulfillment Center i, the sum of the storage size of certain kind of product k should be 
    # less than total.
    C=[]
    for i in I:
        C.append(mod.addConstr(sum(x[i,j,k]*Item.loc[k,"storage_size"] for k in K for j in J)<=FC.loc[i,"capacity"]))
    # Minimize the objective function
    mod.optimize()
    # Store the total minimized transportation cost
    val=pd.DataFrame([mod.objVal],columns=['Objective Value'])
    # Store solution set (how many units of each item stored in each Fulfillment Center that are distributed to a 
    # certain region)
    solution=[]
    for i in I:
        for j in J:
            for k in K:
                solution.append([i,j,k,x[i,j,k].x])
    S=pd.DataFrame(solution,columns=['FC_name',"region_ID","item_ID","shipment"])
    S=S[S["shipment"]!=0]
    # Store the top 5 shadow price set (Increase of limit for certain FC that results in reduce of transportation cost)
    Shadow=[]
    for i in range(len(C)):
        Shadow.append(C[i].pi)
    Shadow_df=pd.DataFrame(Shadow,index=FC.index,columns=["shadow_price"]).sort_values(by="shadow_price")  
    top5=Shadow_df.head()
    
    # Write each dataframe to a different worksheet.
    writer = pd.ExcelWriter('output for '+str(excel), engine='xlsxwriter')
    val.to_excel(writer, sheet_name='Summary',index=False)
    S.to_excel(writer, sheet_name='Solution',index=False)
    top5.to_excel(writer, sheet_name='Capacity Constraints')
    return val, S, top5

In [3]:
val,S,top5=optimize("small_data.xlsx") # test with small data set

Academic license - for non-commercial use only
Optimize a model with 8 rows, 12 columns and 24 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [6e-01, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+03]
Presolve time: 0.02s
Presolved: 8 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.450000e+03   0.000000e+00      0s
       8    3.4007692e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.05 seconds
Optimal objective  3.400769190e+03


In [4]:
val #Minimum Transportation Cost

,Objective Value
0,3400.76919


In [5]:
S #Solution Set

,FC_name,region_ID,item_ID,shipment
1,A,0,1,125.0
2,A,1,0,100.0
3,A,1,1,200.0
5,A,2,1,100.0
6,B,0,0,500.0
7,B,0,1,75.0
10,B,2,0,350.0


In [6]:
top5 # top 5 shadow price set (There are only 2 FC in small data sets)

,shadow_price
FC_name,
B,-0.636208
A,0.000000


In [7]:
val,S,top5=optimize("data.xlsx") # test with large data set

Optimize a model with 9817 rows, 166600 columns and 333200 nonzeros
Coefficient statistics:
  Matrix range     [5e-03, 2e+00]
  Objective range  [2e-03, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+05]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 1937 rows and 32929 columns
Presolve time: 0.43s
Presolved: 7880 rows, 133671 columns, 267342 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.337e+05
 Factor NZ  : 1.421e+05 (roughly 60 MBytes of memory)
 Factor Ops : 2.572e+06 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.13065591e+10  0.00000000e+00  5.95e+07 0.00e+00  4.70e+05     1s
   1   1.23790320e+10 -9.38154348e+07  3.49e+07 5.48e+00  2.80e+05     1s
   2   3.62934680e+09 -1.41723525e+08  9.49e+06 4.93e-01  7.47e+04     1s
   3   5.99820433e+08 -1.34

In [8]:
val #Minimum Transportation Cost

,Objective Value
0,9.841229e+06


In [10]:
S.head(10) #Solution Set

,FC_name,region_ID,item_ID,shipment
1000,SAT1,10,0,2524.0
1001,SAT1,10,1,2485.0
1003,SAT1,10,3,2300.0
1004,SAT1,10,4,3480.0
1005,SAT1,10,5,6208.0
1007,SAT1,10,7,6960.0
1010,SAT1,10,10,4198.0
1011,SAT1,10,11,5078.0
1012,SAT1,10,12,582.0
1013,SAT1,10,13,137.0


In [11]:
top5 # top 5 shadow price set

,shadow_price
FC_name,
TPA1,-5.333088
MKE1,-4.276396
SDF8,-2.486980
BDL1,-2.302010
EWR4,-1.807986
